In [1]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft
import pandas as pd

In [2]:
theta = 0.132328; kappa = 10.980797; sigma = 1.7; rho = -0.351560; v0 = 0.065690
#spot, K = 659.37, 600.0
spot, K = 1.0, 600.0/659.37
#spot, K = 659.37, 758.28
#spot, K = 659.37, 800.0
alp = 0.75
s = np.log(spot)
k = np.log(K)
x = np.log(spot)
feller_cond = 2*kappa*theta > sigma**2
print("Feller condition is ", feller_cond)

Feller condition is  True


In [3]:
def HestonMC (spot, v0, rho, kappa, theta, sigma, T, num_steps, r=0.0, return_log = True):
    # Generate a path
    vt    = np.zeros(num_steps)
    vt[0] = v0
    logSt = np.zeros(num_steps)
    logSt[0] = np.log(spot)
    dt = T / num_steps

    # Milstein scheme for volatility
    for t in range(1,num_steps):
        # Generate random Brownian Motion
        dW_indep = np.random.normal(0.0,1.0,2)
        dW_v = dW_indep[0]
        dW_logS = rho*dW_indep[0] + np.sqrt(1-rho**2)*dW_indep[1] 
        vt[t] = vt[t-1] + kappa*(theta-vt[t-1])*dt + sigma* np.sqrt(vt[t-1]*dt)*dW_v + sigma**2/4.*dt*(dW_v**2-1.)
        if vt[t] < 0.0:
            vt[t] = 0.0
        logSt[t] = logSt[t-1] + (r - vt[t-1]/2.)*dt + np.sqrt(vt[t-1]*dt)*dW_logS

    if return_log:
        return logSt, vt
    else:        
        St= np.exp(logSt)
        return St, vt

In [4]:
u = 1.0 
r = 0.0
#num_paths = 20000
num_paths = 100
payoff_sum = 0.0
T = 1.0
#T = 0.333
num_steps = 252

In [5]:
def get_chf(x_T,u):
    return np.exp(u*1j*x_T)

"""
Compute E[e^{1j*u*x}]
x: log(S)
u: variable for the chf
v0: v0
r: risk-free rate
rho: correlation
ka: kappa
th: theta
sig: sigma
"""
def get_Heston_chf(x, u, v0, r, rho, ka, th, sig, T):
    gam = np.sqrt(sig**2*u*(u+1j) + (ka-1j*rho*sig*u)**2)
    half_gam_T = gam*T/2.0
    coth = 1.0/np.tanh(half_gam_T)
    log_numerator = ka*th*T*(ka-1j*rho*sig*u)/sig**2 + 1j*u*(T*r+x) - (u*(u+1j)*v0)/(gam*coth + ka - 1j*rho*sig*u)
    numerator = np.exp(log_numerator)
    denominator = (np.cosh(half_gam_T) + (ka-1j*rho*sig*u)/gam*np.sinh(half_gam_T))**(2.0*ka*th/sig**2)
    return numerator/denominator

def get_damped_Heston_chf(x, v, v0, r, rho, ka, th, sig, T, alp):
    chf_inp = v - (alp+1.0)*1j
    numerator = np.exp(-r*T) * get_Heston_chf(x, chf_inp, v0, r, rho, ka, th, sig, T)
    denominator = alp**2 + alp - v**2 +1j*(2.0*alp +1.0)*v
    return numerator / denominator

In [6]:
heston_chf_mc = 0.0
mc_call_price = 0.0
for i in range(num_paths):
    logS_last = HestonMC(spot, v0, rho, kappa, theta, sigma, T, num_steps, r=r, return_log = True)[0][-1]
    heston_chf_mc += get_chf(logS_last,u)
    mc_call_price += max(0., np.exp(logS_last)-K)
heston_chf_mc /= num_paths
mc_call_price /= num_paths
print('chf from mc ', heston_chf_mc)
print('analytic chf ', get_Heston_chf(x, u, v0, r, rho, kappa, theta, sigma, T))
print('call from mc ', mc_call_price)
print(spot, K, v0, rho)

chf from mc  (0.9367481614150424-0.05472072214030216j)
analytic chf  (0.9345074136306541-0.055431779426290094j)
call from mc  0.17954005754633623
1.0 0.9099595068019473 0.06569 -0.35156


In [7]:
def get_Heston_fft_call(k, x, v0, r, rho, kappa, theta, sigma, T, alp, N=2**12, dk=0.01):# 2048*5
    u_list = np.array(range(N-1))
    # be careful about scales of dk and dv
#    dk = np.abs(k)*0.01
    dv = 2.0*np.pi/(N*dk)  # N*dv = 2*pi/dk

#    dk = 2.0*np.pi/(N*dv) # nu nu*zeta = 2*pi/N
    
#    print(dv*(N-1))
#    print('dk ds ', dk, ' ',dv)
    k_list = np.array([x + dk*u_elem - N*dk/2.0 for u_elem in u_list])
    v_list = np.array(range(N-1)) * dv
#    print('maxv ', v_list[-1])
#    if np.isinf(np.sinh(v_list[-1]**2)) | np.isinf(np.cosh(v_list[-1]**2)) | np.isinf(np.exp(v_list[-1]**2)):
#        print('Infinite ',v_list[-1]**2)
#        raise Exception('inifite')

    x_list = [] # values in frequency domain
    for (j,vj) in enumerate(v_list): # j=0 to N-1
        if j==0:
            kroneker_delta = 1.0
        else:
            kroneker_delta = 0.0        
        simpson_coeff = dv/3.0*(3.0 + (-1)**j - kroneker_delta)
        xj = np.exp(1j*(N/2.0*dk-s)*vj)*simpson_coeff*get_damped_Heston_chf(x, vj, v0, r, rho, kappa, theta, sigma, T, alp)
        x_list.append(xj)

    fft_prices = np.exp(-alp*k)/np.pi*fft(x_list).real
#    price_index = np.where(k_list == k)[0]
#    fft_price = fft_prices[int(N/2)]
    # interpolate results
    fft_price = np.interp(np.exp(k), np.exp(k_list), fft_prices)
#    print(fft_price)
    return fft_price

In [8]:
#fft_price = get_Heston_fft_call(k, x, v0, r, rho, kappa, theta, sigma, T, alp, N=2048*6)
fft_price = get_Heston_fft_call(k, x, v0, r, rho, kappa, theta, sigma, T, alp, N=2**15, dk=1e-2)
#fft_price = get_Heston_fft_call(np.log(600.0), np.log(659), v0, r, rho, kappa, theta, sigma, T, alp, N=2048*6, dv=1e-3)

C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in tanh


In [9]:
print('call from mc ', mc_call_price)
print('call from FFT ', fft_price)
print('call from FFT ', fft_price*659.37)

call from mc  0.17954005754633623
call from FFT  0.1824234257398621
call from FFT  120.28453423009286


In [10]:
alp

0.75

# Generate Dataset 

In [15]:
num_grid_1d = 6
alp = 0.75
C_price_bounds = [1e-3, 0.5]

k_list = np.linspace(-0.2,0.2,num=num_grid_1d) # logstrike
logS0 = 0.0 # log-spot 
T_list = [1.0] #
v0_list = np.linspace(np.sqrt(1e-1),np.sqrt(0.3),num=num_grid_1d) 
r = 0.0
rho_list = np.linspace(-1.0, 1.0,num=num_grid_1d) 
kappa_list = np.linspace(1e-3, 20.0,num=num_grid_1d) 
theta_list = np.linspace(1e-3, 10.0,num=num_grid_1d) 
sigma_list = np.linspace(1e-3, 20.0,num=num_grid_1d) 

pd_dict = {}
pd_dict['k'] = []; pd_dict['T'] = []; pd_dict['v0'] = []; pd_dict['rho'] = []
pd_dict['kappa'] = []; pd_dict['theta'] = []; pd_dict['sigma'] = []; pd_dict['C_price'] = []
violation_count = 0

for kappa in kappa_list:
    for theta in theta_list:
        for sigma in sigma_list:
#            print(2*kappa*theta - sigma**2)
            feller_cond = 2*kappa*theta > sigma**2
            if feller_cond == True:
                for k in k_list:
                    for T in T_list:
                        for v0 in v0_list:
                            for rho in rho_list:
                                C_price = get_Heston_fft_call(k, logS0, v0, r, rho, kappa, theta, sigma, T, alp, N=2**10, dk=1e-2)
                                if (C_price >= C_price_bounds[0]) | (C_price <= C_price_bounds[1]) | np.isnan(C_price):
                                    continue
                                pd_dict['k'].append(k)
                                pd_dict['T'].append(T)
                                pd_dict['v0'].append(v0)
                                pd_dict['rho'].append(rho)
                                pd_dict['kappa'].append(kappa)
                                pd_dict['theta'].append(theta)
                                pd_dict['sigma'].append(sigma)
                                pd_dict['C_price'].append(C_price)
            else:
                violation_count += 1

df = pd.DataFrame(pd_dict)
print('violations ', violation_count)

C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in cdouble_scalars
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in cdouble_scalars
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in cdouble_scalars
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in exp
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in tanh
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in cosh
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in sinh
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in cdouble_scalars


violations  120


In [16]:
df.dropna(inplace=True)
df.reset_index()
print(df.isna().sum())

k          0
T          0
v0         0
rho        0
kappa      0
theta      0
sigma      0
C_price    0
dtype: int64


In [27]:
df.to_csv('Heston_data', index=False)

In [ ]:
"""
N = 2048*6
u_list = list(range(N-1))
dk = k * 0.002 # zeta
dv = 2.0*np.pi/(N*dk) # nu
k_list = [s + dk*u_elem - N*dk/2.0 for u_elem in u_list]
v_list = range(N-1) * dv
x_list = [] # values in frequency domain
for (j,vj) in enumerate(v_list): # j=0 to N-1
    if j==0:
        kroneker_delta = 1.0
    else:
        kroneker_delta = 0.0        
    coeff = dv/3.0*(3.0 + (-1)**j - kroneker_delta)
    xj = np.exp(1j*(N/2.0*dk-s)*vj)*coeff*get_damped_Heston_chf(x, vj, v0, r, rho, kappa, theta, sigma, T, alp)
    x_list.append(xj)
    
fft_prices = fft(x_list).real
fft_prices *= np.exp(-alp*k)/np.pi
"""